In [ ]:
import pandas as pd
import os
import numpy as np
import json
import regex
import matplotlib.pyplot as plt

In [47]:
bpc=pd.read_excel("BPC_Ingredientes.xlsx")
an3=pd.read_excel('annex_iii_inventory_en_PARA CARGAR en 03.xls')
with open("traducciones.json") as file:
    traducciones=json.load(file)


In [49]:
names=an3['Name'].apply(lambda x: x.lower()).to_list()
ec=an3['EC / List no.'].to_list()
cas=an3['CAS no.'].to_list()



In [50]:
blurb='European Commission Legislation (Regulation (EC) N° 1223/2009 y 2019/831, Annexes I to VI and all the amendments to the cosmetics regulation).'

In [ ]:
bpc_mapped=[]
for n,node in bpc.iterrows():
    for sinonimo in node['Synonyms.formatx'].split(';'):
        buffer=[]
        query=an3[
            (an3['Name'] == sinonimo) |\
            (an3['EC / List no.'] == node['EC No']) |\
            (an3['CAS no.'] == node['CAS No'])
        ]
        if not query.empty:
            node['modificado']=True
            
            node["Info para Reporte"] = (lambda x: traducciones.get(x,x))(node["Info para Reporte"])
            
            ttm=query['Criteria(Indication)'].values[0]
            if node['Anexo.iii.Criteria'] == '-':
                node['Anexo.iii.Criteria']=ttm
                # node['Info para Reporte']=node['Info para Reporte']
                node['Citas']= node['Citas'] + '{};'.format(blurb)
            if node["Info para Reporte"] == "-":
                node["Info para Reporte"] = (lambda x: traducciones.get(x, "-"))(node["Anexo.iii.Criteria"])
            
            if node['CAS No']=='-' and query['CAS no.'].values[0]!='-':
                node['CAS No'] = query['CAS no.'].values[0]
                node['Synonyms.formatx']=node['Synonyms.formatx'] + '{};'.format(query['CAS no.'].values[0])
            if node['EC No']=='-' and query['EC / List no.'].values[0] != '-':
                node['EC No'] = query['EC / List no.'].values[0]
                node['Synonyms.formatx']=node['Synonyms.formatx'] + '{};'.format(query['EC / List no.'].values[0])
            buffer=node    
           
            break
            
        else:
            buffer=node
        
    bpc_mapped.append(buffer)


            
            

In [ ]:
df_an3_no_dd=pd.concat(bpc_mapped, axis=1).T

In [ ]:
df_an3_no_dd.to_excel("prueba_es_este.xlsx")

In [ ]:
df_an3_2=pd.concat(bpc_mapped, axis=1).T.drop_duplicates(subset=['Synonyms.formatx'])


In [ ]:
risky=df_an3_no_dd[df_an3_no_dd['Total.Risk']>=2]
risky_itr=risky[risky['Info para Reporte']!='-']
risky_final=risky_itr[risky_itr.Citas!='-']
risky_final.to_excel("risky_prueba.xlsx")

In [ ]:
risky.to_excel("removiendo_harmonised.xlsx")

In [19]:
risky.to_excel("rararara.xlsx")

In [11]:
risky_final.to_excel('bpc_con_an3_itr_citas.xlsx')

In [12]:
extractos=pd.read_excel('extractos.xlsx')
extractos.columns=['name', 'Info para Reporte', 'Citas']

In [13]:
rows_con_extractos=[]
for n,node in bpc.iterrows():
    for sinonimo in node['Synonyms.formatx'].split(';'):
        query=extractos[
            (extractos['name'] == sinonimo)
        ]
        if query.empty:
            rows_con_extractos.append(node)
        else:
            print('salte aca')
            node['modificado']=True
            ipr=query['Info para Reporte'].values[0]
            citas=query['Citas'].values[0]
            if node['Info para Reporte'] == '-':
                # node['Info para Reporte']=ipr
                node['Info para Reporte']=node['Info para Reporte']
                node['Citas']=node['Citas']+ ' {}'.format(citas)
            rows_con_extractos.append(node)

            break
            

Este script está actualizado, si lo que quiero es la tabla que se le terminó mandando a roberto lo que necesito es entonces solo la variable risky!